## Load Packages

In [1]:
        
include("printmat.jl")

printlnPs (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
    default(show=false)               #for pyplot: avoids pop-ups
else    
    gr(size=(600,400))
    default(show=true)
end

# Predictions from an AR(1)

Consider an AR(1) with mean $\mu$

$r_{t+1}-\mu=\rho\left(  r_{t}-\mu\right)  +\varepsilon_{t+1}$

The forecast for $t+s$ is

$\text{E}_{t}r_{t+s}=\left(  1-\rho^{s}\right)  \mu+\rho ^{s}r_{t}$


In [3]:
function AR1Prediction(x0,ρ,μ,s)                       #function for calculating forecast from AR(1)
    E0xs = (1-ρ^s)*μ + ρ^s*x0
    return E0xs
end

AR1Prediction (generic function with 1 method)

In [4]:
sMax = 120
sM = 1:sMax

ρ = 0.975
μ = 0.05
xPred = fill(NaN,(sMax,3))                              #predictions[horizon,today's value]
for s in sM
    xPred[s,:] = AR1Prediction([0.03 0.05 0.07],ρ,μ,s)
end    

plot(sM,xPred[:,1],color=:black,linewidth=2,label="3%")
plot!(sM,xPred[:,2],color=:red,linewidth=2,label="5%")
plot!(sM,xPred[:,3],color=:blue,linewidth=2,label="7%")
title!("Forecast of future short interest rates (based on different starting values), (mu,rho) = ($μ,$ρ)"),
xlabel!("forecast horizons (months)")
ylabel!("short interest rate")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 120 
 
 
 0.04 
 
 
 0.05 
 
 
 0.06 
 
 
 Forecast of future short interest rates (based on different starting values), (mu,rho) = (0.05,0.975) 
 
 
 forecast horizons (months) 
 
 
 short interest rate 
 
<polyline clip-path="url(#clip02)" style="stroke:#00002d; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 54.7703,369.674 59.319,365.345 63.8677,361.125 68.4164,357.01 72.9651,352.998 77.5137,349.086 82.0624,345.272 86.6111,341.554 91.1598,337.928 95.7084,334.393 
 100.257,330.946 104.806,327.586 109.354,324.309 113.903,321.115 118.452,318 123.001,314.963 127.549,312.003 132.098,309.116 136.647,306.301 141.195,303.557 
 145.744,300.881 150.293,298.272 154.841,295.728 159.39,293.248 163.939,290.83 168.487,288.473 173.036,286.174 177.585,283.933 182.133,281.748 186.682,279.617 
 191.231,277.54 195.779,275.515 200.328,273.54 204.877,271.615 209.425,269.738 213.974,267.907 218.523,266.123 223.071,264.383 227.62,262.687 232.169,261.033 
 236.717,259.42 241.266,257.848 245.815,256.315 250.363,254.82 254.912,253.363 259.461,251.942 264.01,250.557 268.558,249.206 273.107,247.889 277.656,246.605 
 282.204,245.353 286.753,244.132 291.302,242.942 295.85,241.782 300.399,240.651 304.948,239.547 309.496,238.472 314.045,237.423 318.594,236.401 323.142,235.404 
 327.691,234.432 332.24,233.485 336.788,232.561 341.337,231.66 345.886,230.782 350.434,229.925 354.983,229.09 359.532,228.276 364.08,227.483 368.629,226.709 
 373.178,225.954 377.726,225.218 382.275,224.501 386.824,223.802 391.372,223.12 395.921,222.455 400.47,221.807 405.019,221.175 409.567,220.559 414.116,219.958 
 418.665,219.372 423.213,218.801 427.762,218.244 432.311,217.701 436.859,217.172 441.408,216.656 445.957,216.153 450.505,215.662 455.054,215.184 459.603,214.717 
 464.151,214.263 468.7,213.819 473.249,213.387 477.797,212.966 482.346,212.555 486.895,212.154 491.443,211.763 495.992,211.382 500.541,211.011 505.089,210.649 
 509.638,210.296 514.187,209.952 518.735,209.616 523.284,209.289 527.833,208.97 532.382,208.659 536.93,208.356 541.479,208.06 546.028,207.772 550.576,207.49 
 555.125,207.216 559.674,206.949 564.222,206.689 568.771,206.435 573.32,206.187 577.868,205.945 582.417,205.71 586.966,205.48 591.514,205.257 596.063,205.038 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 54.7703,196.528 59.319,196.528 63.8677,196.528 68.4164,196.528 72.9651,196.528 77.5137,196.528 82.0624,196.528 86.6111,196.528 91.1598,196.528 95.7084,196.528 
 100.257,196.528 104.806,196.528 109.354,196.528 113.903,196.528 118.452,196.528 123.001,196.528 127.549,196.528 132.098,196.528 136.647,196.528 141.195,196.528 
 145.744,196.528 150.293,196.528 154.841,196.528 159.39,196.528 163.939,196.528 168.487,196.528 173.036,196.528 177.585,196.528 182.133,196.528 186.682,196.528 
 191.231,196.528 195.779,196.528 200.328,196.528 204.877,196.528 209.425,196.528 213.974,196.528 218.523,196.528 223.071,196.528 227.62,196.528 232.169,196.528 
 236.717,196.528 241.266,196.528 245.815,196.528 250.363,196.528 254.912,196.528 259.461,196.528 264.01,196.528 268.558,196.528 273.107,196.528 277.656,196.528 
 282.204,196.528 286.753,196.528 291.302,196.528 295.85,196.528 300.399,196.528 304.948,196.528 309.496,196.528 314.045,196.528 318.594,196.528 323.142,196.528 
 327.691,196.528 332.24,196.528 336.788,196.528 341.337,196.528 345.886,196.528 350.434,196.528 354.983,196.528 359.532,196.528 364.08,196.528 368.629,196.528 
 373.178,196.528 377.726,196.528 382.275,196.528 386.824,196.528 391.372,196.528 395.921,196.528 400.47,196.528 405.019,196.528 409.567,196.528 414.116,196.528 
 418.665,196.528 423.213,196.528 427.762,196.528 432.311,196.528 436.859,196.528 441.408,196.528 445.957,196.528 450.505,196.528 455.054,196.528 459.603,196.528 
 464.151,196.528 468.7,196.528

# The Vasicek Model

The Vasicek model (without risk premia) implies that 

$\begin{align}
y_{t}(m)  &  =a(m)+b(m)r_{t} \ \text{, where } \\
a(m)  &  = \mu\left[  1-b(m)\right] \ \text{ and}\\
b(m)  &  =(1-\rho^{m})/[(1-\rho)m].
\end{align}$

In [5]:
function Vasicek(r,ρ,μ,m)                   #Vasicek model -> interest rate and (a,b) coeffs
    if ρ == 1.0
        b = 1.0
    else    
        b = (1-ρ^m)/((1-ρ)*m)
    end    
    a = μ*(1-b)
    y = a + b*r
    return y,a,b
end

Vasicek (generic function with 1 method)

In [6]:
yM = fill(NaN,(sMax,3))                             #interest rate[maturity,today's short rate]
ab = fill(NaN,(sMax,2))
for m in sM
    (y,a,b) = Vasicek([0.03 0.05 0.07],ρ,μ,m)
    yM[m,:] = y
    ab[m,:] = [a b]
end    

In [7]:
plot(sM,yM[:,1],color=:black,linewidth=2,label="3%")
plot!(sM,yM[:,2],color=:red,linewidth=2,label="5%")
plot!(sM,yM[:,3],color=:blue,linewidth=2,label="7%")
title!("Vasicek model: interest rates (based on different current short rates), (mu,rho) = ($μ,$ρ)"),
xlabel!("maturity (months)")
ylabel!("interest rate")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 120 
 
 
 0.03 
 
 
 0.04 
 
 
 0.05 
 
 
 0.06 
 
 
 0.07 
 
 
 Vasicek model: interest rates (based on different current short rates), (mu,rho) = (0.05,0.975) 
 
 
 maturity (months) 
 
 
 interest rate 
 
<polyline clip-path="url(#clip02)" style="stroke:#00002d; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 54.7703,369.674 59.319,367.51 63.8677,365.382 68.4164,363.289 72.9651,361.231 77.5137,359.206 82.0624,357.216 86.6111,355.258 91.1598,353.333 95.7084,351.439 
 100.257,349.576 104.806,347.743 109.354,345.941 113.903,344.167 118.452,342.423 123.001,340.707 127.549,339.018 132.098,337.357 136.647,335.722 141.195,334.114 
 145.744,332.532 150.293,330.974 154.841,329.442 159.39,327.934 163.939,326.45 168.487,324.989 173.036,323.551 177.585,322.137 182.133,320.744 186.682,319.373 
 191.231,318.024 195.779,316.695 200.328,315.387 204.877,314.1 209.425,312.832 213.974,311.585 218.523,310.356 223.071,309.146 227.62,307.955 232.169,306.782 
 236.717,305.627 241.266,304.489 245.815,303.369 250.363,302.265 254.912,301.179 259.461,300.108 264.01,299.054 268.558,298.015 273.107,296.992 277.656,295.985 
 282.204,294.992 286.753,294.014 291.302,293.05 295.85,292.101 300.399,291.165 304.948,290.244 309.496,289.335 314.045,288.44 318.594,287.558 323.142,286.689 
 327.691,285.832 332.24,284.988 336.788,284.156 341.337,283.336 345.886,282.527 350.434,281.73 354.983,280.944 359.532,280.17 364.08,279.406 368.629,278.653 
 373.178,277.911 377.726,277.179 382.275,276.458 386.824,275.746 391.372,275.045 395.921,274.353 400.47,273.67 405.019,272.997 409.567,272.333 414.116,271.679 
 418.665,271.033 423.213,270.396 427.762,269.768 432.311,269.148 436.859,268.536 441.408,267.933 445.957,267.338 450.505,266.751 455.054,266.171 459.603,265.6 
 464.151,265.035 468.7,264.479 473.249,263.929 477.797,263.387 482.346,262.852 486.895,262.324 491.443,261.803 495.992,261.288 500.541,260.78 505.089,260.279 
 509.638,259.784 514.187,259.296 518.735,258.813 523.284,258.337 527.833,257.867 532.382,257.403 536.93,256.944 541.479,256.492 546.028,256.045 550.576,255.603 
 555.125,255.167 559.674,254.737 564.222,254.312 568.771,253.892 573.32,253.477 577.868,253.067 582.417,252.662 586.966,252.263 591.514,251.868 596.063,251.477 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 54.7703,196.528 59.319,196.528 63.8677,196.528 68.4164,196.528 72.9651,196.528 77.5137,196.528 82.0624,196.528 86.6111,196.528 91.1598,196.528 95.7084,196.528 
 100.257,196.528 104.806,196.528 109.354,196.528 113.903,196.528 118.452,196.528 123.001,196.528 127.549,196.528 132.098,196.528 136.647,196.528 141.195,196.528 
 145.744,196.528 150.293,196.528 154.841,196.528 159.39,196.528 163.939,196.528 168.487,196.528 173.036,196.528 177.585,196.528 182.133,196.528 186.682,196.528 
 191.231,196.528 195.779,196.528 200.328,196.528 204.877,196.528 209.425,196.528 213.974,196.528 218.523,196.528 223.071,196.528 227.62,196.528 232.169,196.528 
 236.717,196.528 241.266,196.528 245.815,196.528 250.363,196.528 254.912,196.528 259.461,196.528 264.01,196.528 268.558,196.528 273.107,196.528 277.656,196.528 
 282.204,196.528 286.753,196.528 291.302,196.528 295.85,196.528 300.399,196.528 304.948,196.528 309.496,196.528 314.045,196.528 318.594,196.528 323.142,196.528 
 327.691,196.528 332.24,196.528 336.788,196.528 341.337,196.528 345.886,196.528 350.434,196.528 354.983,196.528 359.532,196.528 364.08,196.528 368.629,196.528 
 373.178,196.528 377.726,196.528 382.275,196.528 386.824,196.528 391.372,196.528 395.921,196.528 400.47,196.528 405.019,196.528 409.567,196.528 414.116,196.528 
 418.665,196.528 423.213,196.528 427.762,196.528 432.311,196.528 436.859,196.528 441.408,196.528 445.957,196.528 450.505,196.528 455.054,196.528 459.603,196.528 
 464.151,196.52

In [8]:
plot(sM,ab[:,1],color=:black,linewidth=2,label="a")
plot!(sM,ab[:,2],color=:red,linewidth=2,label="b")
title!("Vasicek model: a and b in y(m) = a(m) + b(m)r, (mu,rho) = ($μ,$ρ)")
xlabel!("maturity (months)")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 20 
 
 
 40 
 
 
 60 
 
 
 80 
 
 
 100 
 
 
 120 
 
 
 0.0 
 
 
 0.2 
 
 
 0.4 
 
 
 0.6 
 
 
 0.8 
 
 
 1.0 
 
 
 Vasicek model: a and b in y(m) = a(m) + b(m)r, (mu,rho) = (0.05,0.975) 
 
 
 maturity (months) 
 
<polyline clip-path="url(#clip02)" style="stroke:#00002d; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 30.6037,369.674 35.3554,369.458 40.1072,369.245 44.859,369.036 49.6107,368.83 54.3625,368.627 59.1142,368.428 63.866,368.233 68.6177,368.04 73.3695,367.851 
 78.1213,367.664 82.873,367.481 87.6248,367.301 92.3765,367.123 97.1283,366.949 101.88,366.777 106.632,366.609 111.384,366.442 116.135,366.279 120.887,366.118 
 125.639,365.96 130.391,365.804 135.142,365.651 139.894,365.5 144.646,365.352 149.398,365.206 154.149,365.062 158.901,364.92 163.653,364.781 168.405,364.644 
 173.156,364.509 177.908,364.376 182.66,364.245 187.412,364.117 192.163,363.99 196.915,363.865 201.667,363.742 206.419,363.621 211.171,363.502 215.922,363.385 
 220.674,363.269 225.426,363.156 230.178,363.044 234.929,362.933 239.681,362.825 244.433,362.718 249.185,362.612 253.936,362.508 258.688,362.406 263.44,362.305 
 268.192,362.206 272.943,362.108 277.695,362.012 282.447,361.917 287.199,361.823 291.95,361.731 296.702,361.64 301.454,361.551 306.206,361.463 310.957,361.376 
 315.709,361.29 320.461,361.205 325.213,361.122 329.964,361.04 334.716,360.959 339.468,360.88 344.22,360.801 348.972,360.724 353.723,360.647 358.475,360.572 
 363.227,360.498 367.979,360.425 372.73,360.352 377.482,360.281 382.234,360.211 386.986,360.142 391.737,360.074 396.489,360.006 401.241,359.94 405.993,359.875 
 410.744,359.81 415.496,359.746 420.248,359.683 425,359.621 429.751,359.56 434.503,359.5 439.255,359.44 444.007,359.382 448.758,359.324 453.51,359.267 
 458.262,359.21 463.014,359.155 467.765,359.1 472.517,359.045 477.269,358.992 482.021,358.939 486.773,358.887 491.524,358.836 496.276,358.785 501.028,358.735 
 505.78,358.685 510.531,358.636 515.283,358.588 520.035,358.54 524.787,358.493 529.538,358.447 534.29,358.401 539.042,358.356 543.794,358.311 548.545,358.267 
 553.297,358.223 558.049,358.18 562.801,358.138 567.552,358.096 572.304,358.054 577.056,358.013 581.808,357.973 586.559,357.933 591.311,357.893 596.063,357.854 
 
 "/>
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 30.6037,23.3815 35.3554,27.7101 40.1072,31.9666 44.859,36.1523 49.6107,40.2686 54.3625,44.3167 59.1142,48.2979 63.866,52.2134 68.6177,56.0646 73.3695,59.8525 
 78.1213,63.5783 82.873,67.2433 87.6248,70.8485 92.3765,74.395 97.1283,77.8839 101.88,81.3164 106.632,84.6933 111.384,88.0158 116.135,91.2849 120.887,94.5014 
 125.639,97.6665 130.391,100.781 135.142,103.846 139.894,106.862 144.646,109.83 149.398,112.752 154.149,115.627 158.901,118.457 163.653,121.242 168.405,123.984 
 173.156,126.683 177.908,129.339 182.66,131.955 187.412,134.53 192.163,137.065 196.915,139.561 201.667,142.018 206.419,144.438 211.171,146.82 215.922,149.166 
 220.674,151.476 225.426,153.752 230.178,155.992 234.929,158.199 239.681,160.373 244.433,162.513 249.185,164.622 253.936,166.699 258.688,168.745 263.44,170.76 
 268.192,172.746 272.943,174.702 277.695,176.629 282.447,178.528 287.199,180.399 291.95,182.242 296.702,184.059 301.454,185.849 306.206,187.613 310.957,189.352 
 315.709,191.065 320.461,192.754 325.213,194.418 329.964,196.058 334.716,197.675 339.468,199.269 344.22,200.841 348.972,202.39 353.723,203.917 358.475,205.423 
 363.227,206.907 367.979,208.371 372.73,209.814 377.482,211.237 382.234,212.641 386.986,214.025 391.737,215.389 396.489,216.735 401.241,218.063 405.993,219.372 
 410.744,220.664 415.496,221.938 420.248,223.194 425,224.434 429.751,225.657 434.503,226.864 439.255,228.054 444.007,229.228 448.758,230.387 453.51,231.531 
 458.262,232.659 463.014,233.772 467.765,234.871 472.517,235.955 477.269,237.

# Hedging Using the Vasicek Model

Recall that the change of the (value of the) hedge portfolio is 

$\begin{equation}
\Delta V=h\Delta P_{H}-\Delta P_{L}
\end{equation}$


1. For an initial value of the short log interest rate $r$, use the Vasicek model to calculate all spot rates $y(m)$ for $m$ starting at 1 and going until the maturity of the liability or hedge (whichever is longest).

2. Use the spot interest rates ($y(m)$) to calculate the prices of both the liability and the hedge bond.

3. Redo points 1 and 2, but starting from a somewhat different short rate.
    
4. Calculate the $h$ value that makes $\Delta V = 0$, that is, $h=\Delta P_{L}/\Delta P_{H}$.

Hint: to transform from log rates $y(m)$ to effective rates $Y(m)$, use $Y(m) = \exp[y(m)]-1$.

### From Chapter on Bonds 1

In [9]:
function BondPrice3Ps(Y,cf,mk)              #cf is a vector of all cash flows, mk are years
    cfac = cf./((1+Y).^mk)                  #c/(1+y1)^t1 + c/(1+y2)^t2 + ...+ c/(1+y2)^m
    P    = sum(cfac)                        #price
    return P
end

BondPrice3Ps (generic function with 1 method)

For given parameters of the Vasicek model ($\rho,\mu$), calculate bond prices at the short rate is $r_0$ and then at $r_1$. You may want to repeat the calculations but at different values of $\rho$

In [10]:
ρ = 0.9999                       #EXPERIMENT WITH CHANGING THIS
μ = 0.05

cfL = repeat([zeros(11);0.2],outer=10)   #cash flow of liability, MONTHLY
mkL = 1:120                              #times of the cash flows
cfH = [zeros(35);1]                      #cash flow of hedge bond
mkH = 1:36

r₀ = 0.05                         #initial (day 0) short log interest rate
r₁ = 0.049                        #after change (day 1)

y₀ = fill(NaN,length(mkL))        #yield curve, before
y₁ = fill(NaN,length(mkL))        #yield curve, after
for m in mkL                        
    ym,   = Vasicek([r₀ r₁],ρ,μ,m)
    y₀[m] = ym[1]
    y₁[m] = ym[2]
end    

PL₀ = BondPrice3Ps(exp(y₀)-1,cfL,mkL/12)       #liability, before, /12 to get years
PL₁ = BondPrice3Ps(exp(y₁)-1,cfL,mkL/12)       #after 
ΔPL = PL₁ - PL₀
printlnPs("PL₀, PL₁, ΔPL: ",[PL₀ PL₁ ΔPL])

PH₀ = BondPrice3Ps(exp(y₀[mkH])-1,cfH,mkH/12)                   #hedge bond
PH₁ = BondPrice3Ps(exp(y₁[mkH])-1,cfH,mkH/12)                   
ΔPH = PH₁ - PH₀
printlnPs("\nPH₀, PH₁, ΔPH: ",[PH₀ PH₁ ΔPH])

h = ΔPL/ΔPH
hdollars = h*PH₀/PL₀

printlnPs("\nh, dollars in hedge bond/value of liability: ", h," ",hdollars)

PL₀, PL₁, ΔPL:      1.535     1.543     0.008

PH₀, PH₁, ΔPH:      0.861     0.863     0.003

h, dollars in hedge bond/value of liability:      3.024      1.696
